In [ ]:
#| default_exp Dataset

In [ ]:
#|hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| export 
import fastcore.all as fc
import random
import torch.multiprocessing as mp

In [ ]:
#| export
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import torchvision


In [ ]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [ ]:
#| export
class Dataset:
    def __init__(self, x, y): fc.store_attr()
        
    def __len__(self): return len(self.x)
    
    def __getitem__(self,i): return self.x[i], self.y[i]
    
    def split(self, percent=0.2):
        res = list(range(len(self.x)))
        random.shuffle(res)
        sp=int(len(self.x)*0.2)
        return Dataset(self.x[res[sp:]],self.y[res[sp:]]),Dataset(self.x[res[:sp]],self.y[res[:sp]])

In [ ]:
#| export
def collate(b):
    xs,ys = zip(*b)
    return torch.stack(xs),torch.stack(ys)

In [ ]:
#| export
class Sampler():
    def __init__(self, ds, shuffle=False): self.n,self.shuffle = len(ds),shuffle
    def __iter__(self):
        res = list(range(self.n))
        if self.shuffle: random.shuffle(res)
        return iter(res)

In [ ]:
#| export
class BatchSampler():
    def __init__(self, sampler, bs, drop_last=False): fc.store_attr()
    def __iter__(self): yield from fc.chunked(iter(self.sampler), self.bs, drop_last=self.drop_last)

In [ ]:
#| export
class DataLoader():
    def __init__(self, ds, batchs, n_workers=1, collate_fn=collate): fc.store_attr()
    def __iter__(self):
        yield from map(self.ds.__getitem__, iter(self.batchs))
     

In [ ]:
#| export 

def get_dataset():
    data = datasets.FashionMNIST(root="data",train=True,download=True,transform=ToTensor())
    data = Dataset(torch.flatten(data.data, start_dim=1).float(),data.targets)
    t,v=data.split()
    td=DataLoader(t, BatchSampler(Sampler(t,True),64))
    vd=DataLoader(v, BatchSampler(Sampler(v),2*64))
    return td, vd

In [ ]:
#|export
def accuracy(out, yb): return (out.argmax(dim=1)==yb).float().mean()

In [ ]:
import nbdev; nbdev.nbdev_export()